In [1]:
# from pandas_datareader import data
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np
# from dash import Dash, html, dcc
# import plotly.express as px
# import pandas as pd
# from dash.dependencies import Input, Output, State
# import dash_cool_components
# import dash
# import dash_bootstrap_components as dbc
# import datetime

In [18]:
#import dash_renderer
#!pip install dash-renderer

In [3]:
# app = dash.Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
from pandas_datareader import data
import matplotlib.pyplot as plt
import numpy as np
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output, State, MATCH, ALL
import dash_cool_components
import dash
import dash_renderer
import dash_bootstrap_components as dbc
import datetime
import plotly.graph_objects as go
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

    
app.layout = html.Div(
    [
        html.H1('Stock Funds Analysis', style = {'textAlign': 'center'}),
#         dbc.Row([dbc.Col(html.H3("Stock Analysis"),
#                        width= {'size':6, 'offset':3},
#                        )]
#                ),
        
        dbc.Row([
                dbc.Col(
                        dcc.Input(id="input", value="", placeholder="add value..."),
                        
                        ),
                dbc.Col(
                        html.Button("Add Option", id="submit", n_clicks=0),
                        ),
                dbc.Col(
                        dcc.Dropdown([
                        {}], multi = True, id="dropdown", persistence = True),
                         width = {'size':3, 'offset' : 2},
                        ),
                       

                dbc.Col(
                        dbc.Button("Show Funds", id="my-button", n_clicks=0),
                        style = {"paddingLeft" : 0},
                )
                ]),
                
            
            dcc.Graph(id='my_graph',
            figure={}), 
        
            html.Div(id="slider", children = []),
            html.Div(id="slider_output", children = []),
            html.Div(id="slider_value", children = []),
                
     dcc.Store(id='store-data',data = [], storage_type ='session')       
     #Should add a div to store df results, so do not to keep calling           
    ]
)

@app.callback(
    Output('store-data','data'),
    Input('dropdown','value')
)
def store_data(value):
    return value

#this callback is for: Adding user input into the dropdown list
@app.callback(
    Output("dropdown", "options"),
    Input("submit", "n_clicks"),
    #State("store-data",'data'),
    State("input", "value"),
    State("dropdown", "options"),
    prevent_initial_call=True,
)
def add_dropdown_option(n_clicks, input_value, options):
    return options + [{"label": input_value, "value": input_value}]

#this callback is for: displaying the graph based on user input
@app.callback(
    [Output(component_id='my_graph', component_property='figure'),
    Output('slider','children')],
    [Input('my-button','n_clicks')],
    [State('store-data', 'data')],
    #[State('dropdown','value')], 
    prevent_initial_call=True
)
def display_test(n,input_data):
    fig = go.Figure()
    start = '2017-01-01'
    end = '2021-12-31'
    df = data.DataReader(input_data, 'yahoo', start, end)
    adj_close = df["Adj Close"]
    ret_series = ((1 + adj_close.pct_change()).cumprod() -1)
    fig = px.line(ret_series, y=ret_series.columns, title='Life expectancy in Canada')
    
    children = []
    
    for i in range (len(input_data)):
                    new_label = html.Label(input_data[i])
                    new_slider = dcc.Slider(
                    0.1,1,0.1, id={
                                    'type': 'slider',
                                    'index': "hu"
                    }, persistence = True) #id= input_data1[i]
                    children.append(new_label)
                    children.append(new_slider)
    
    return [fig,children]

@app.callback(
    [Output('slider_output', 'children'),
    Output('slider_value','children')],
    Input({'type': 'slider', 'index': ALL}, 'value'),
    #Input("store-data",'data'), 
    [State('dropdown','value')],
    prevent_initial_call = True
    
)
def sharpe(values,dropdown):
        start_date = '2016-01-01'
        end_date = '2017-12-31'
        df = data.DataReader(dropdown,'yahoo', start_date, end_date)
        #Use loc method to ensure we do not consider extra days of data
        df = df.loc['2016-1-1': '2017-12-31']
        df_closes = df['Adj Close']
        returns = df_closes.pct_change().dropna()
        mean_returns = returns.mean()
        cov_matrix = returns.cov()
        num_portfolios = 2500
        risk_free_rate = 0.0178
        
        adj_close = df["Adj Close"]
        #risk_free_ann_ret_rate = 0.01 #to be changed
        returns_ts = adj_close.pct_change().dropna()
        avg_daily_ret = returns_ts.mean()

        returns_ts['RiskFree_Rate'] = risk_free_rate/252
        avg_rf_ret = returns_ts['RiskFree_Rate'].mean()

        #calculate individualsharpe ratio
        result = []
        for i in range (len(dropdown)):
        #Add the excess return columns for each ETF
            returns_ts['Excess_ret_' + dropdown[i]] = returns_ts[dropdown[i]] - returns_ts['RiskFree_Rate']
            sharpe = ((mean_returns[dropdown[i]] - avg_rf_ret) /returns_ts['Excess_ret_' + dropdown[i]].std())*np.sqrt(252)
            result.append(dropdown[i])
            result.append(html.Br())
            result.append(sharpe)
            result.append(html.Br())  
            
        # Combination with user's weight value
        #user_weight = values
        
        
        
        
        
        #Optimization
        an_vol = np.std(returns) * np.sqrt(252)
        an_rt = mean_returns * 252
        results = np.zeros((3,num_portfolios))
        weights_record = []
        for i in range(num_portfolios):
                weights = np.random.random(len(dropdown))
                weights /= np.sum(weights)
                weights_record.append(np.round(weights,2))
                returns = np.sum(mean_returns*weights ) *252

                std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
                portfolio_std_dev, portfolio_return = std, returns
                results[0,i] = std
                results[1,i] = returns
                results[2,i] = (returns - risk_free_rate) / std #sharpe ratio
            #return results,weights_record
        #results, weights = random_portfolios(num_portfolios,mean_returns, cov_matrix, risk_free_rate)

        max_sharpe_idx = np.argmax(results[2])
        sdp, rp = results[0,max_sharpe_idx], results[1,max_sharpe_idx]
        max_sharpe_allocation = pd.DataFrame(weights_record[max_sharpe_idx],index=df_closes.columns,columns=['allocation'])
        max_sharpe_allocation.allocation = [round(i,2)for i in max_sharpe_allocation.allocation]
        max_sharpe_allocation = max_sharpe_allocation.T

        min_vol_idx = np.argmin(results[0])
        sdp_min, rp_min = results[0,min_vol_idx], results[1,min_vol_idx]
        min_vol_allocation = pd.DataFrame(weights_record[min_vol_idx],index=df_closes.columns,columns=['allocation'])
        min_vol_allocation.allocation = [round(i,2)for i in min_vol_allocation.allocation]
        min_vol_allocation = min_vol_allocation.T

        print ("-"*80)
        print ("Maximum Sharpe Ratio Portfolio Allocation\n")
        print ("Annualised Return:", round(rp,2))
        print ("Annualised Volatility:", round(sdp,2))
        print ("\n")
        print (max_sharpe_allocation)
        print(results[2,max_sharpe_idx])
        print ("-"*80)
        print ("Minimum Volatility Portfolio Allocation\n")
        print ("Annualised Return:", round(rp_min,2))
        print ("Annualised Volatility:", round(sdp_min,2))
        print ("\n")
        print (min_vol_allocation)
        print(results[2,min_vol_idx])
        print ("\n")
        print(weights_record[max_sharpe_idx])
        ha = weights_record[max_sharpe_idx]
        output = []
        for i in range (len(weights_record[max_sharpe_idx])):
            output.append(weights_record[max_sharpe_idx][i])
            output.append(html.Br())
        return [result,output]

        #an_vol = np.std(returns) * np.sqrt(252)
        #an_rt = mean_returns * 252
#         fig, ax = plt.subplots(figsize=(10, 7))
#         ax.scatter(an_vol,an_rt,marker='o',s=200)

#         for i, txt in enumerate(df_closes.columns):
#             ax.annotate(txt, (an_vol[i],an_rt[i]), xytext=(10,0), textcoords='offset points')
#         ax.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
#         ax.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')

#         ax.scatter(results[0,:],results[1,:],c=results[2,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)
#         #    ax.colorbar()
#         #    ax.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
#         #    ax.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')
#         ax.set_title('Simulated Portfolio Optimization based on Efficient Frontier')
#         ax.set_xlabel('annualised volatility')
#         ax.set_ylabel('annualised returns')
#         ax.legend(labelspacing=0.8)
                
#         start = '2017-01-01'
#         end = '2021-12-31'
#         df1 = data.DataReader(dropdown, 'yahoo', start, end)

        
# Date to be changed to user input
# Trying to only call data reader once by saving the result in a hidden div
#Have not done 
# Sharpe ratio -> Have not implemented to sum up all slider to 1.0


    
    
if __name__ == '__main__':
    app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Mar/2022 20:17:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 20:17:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 20:17:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 20:17:02] "GET /_favicon.ico?v=2.2.0 HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 20:17:02] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 20:17:02] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 20:17:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 20:17:02] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 20:17:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 20:17:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 20:17:15] "POST /_dash-update-component HTTP/1.1" 

--------------------------------------------------------------------------------
Maximum Sharpe Ratio Portfolio Allocation

Annualised Return: 0.25
Annualised Volatility: 0.18


Symbols     AAPL  GOOGL  TSLA
allocation  0.66   0.26  0.08
1.3218488106271282
--------------------------------------------------------------------------------
Minimum Volatility Portfolio Allocation

Annualised Return: 0.22
Annualised Volatility: 0.16


Symbols     AAPL  GOOGL  TSLA
allocation  0.32   0.62  0.06
1.216409992314114


[0.66 0.26 0.08]


In [73]:
#OLD

import dash_cool_components
import dash
from dash.dependencies import Input, Output
import dash_html_components as html
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
from pandas_datareader import data as web
import datetime

app = dash.Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    dbc.Row(
        dbc.Col(html.H1("Stock Market Dashboard", className='text-center text-primary mb-4'),width=12)
    ),
    dbc.Row(
        dbc.Col([dash_cool_components.TagInput(
                id="input"),
                 html.Button('Submit', id='submit-val', n_clicks=0),
                html.Div(id='output-graph')])
                
        
    ),
    dcc.Graph(id='my-graph')
])



# @app.callback(
#     Output('output', 'value'),
#     [Input('input', 'value')]
# )
# def timezone_test(value):
#     if value:
#         tags = [e['displayValue'] for e in value]
#         return tags
    
@app.callback(
    Output('my-graph', 'figure'),
    Input('submit-val', 'n_clicks'),
    [State('input', 'value')],
    prevent_initial_call=True
)
def display_test(asd,value):
    if value:
        tags = [e['displayValue'] for e in value]
        start = datetime.datetime(2015, 1, 1)
        end = datetime.datetime.now()
        df = web.DataReader(tags, 'yahoo', start, end)
    return {
        'data': [{
            'x': df.index,
            'y': df.Close
        }],
        'layout': {'margin': {'l': 40, 'r': 0, 't': 20, 'b': 30}}
    }    

#     return dcc.Graph(
#         id='example-graph',
#         figure={
#             'data': [
#                 {'x': df.index, 'y': df.Close, 'type': 'line', 'name': tags},
#             ],
#             'layout': {
#                 'title': tags
#             }
#         }
#     )
        
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
